<a href="https://colab.research.google.com/github/MinGyeongdo/Sesion2_projcet/blob/main/Predict_grosses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 해결 주제
### 최근 2년이내에 개봉한 영화의 지금까지 통계결과에 의한 총 매출 예측하기.
> 제작한 영화의 앞으로의 수요를 예측하는 것은 경제적인 측면에서 매우 중요한 일입니다.<br> 앞으로 얼마나 팔릴까? 라는 질문은 여러부분에서 중요하게 사용됩니다.<br>

- 첫째, 영화 상영관의 배정<br>
당연하게 많이 보고 매출이 많이 나올 것 같은 영화는 많은 관객을 수용하기 위해 많은 상영관과 시간을 배정해야합니다. 반대로 매출이 별로 나오지 않는 영화 같은 경우에는 상영관을 축소시켜 기업입장에서 최대한 효율적인 방향으로 비용을 사용해야 합니다.<br><br>

- 둘째, 매출을 통한 마케팅 전략 수립<br>
 만들 모델의 한계점이기도 하지만, 이 모델의 인사이트를 통해 다양한 마케팅 전략을 수립할 수 있습니다. 영화의 인기도에 미치는 Feature는 굉장히 많습니다. 최근에 개봉한 탑건 같은 경우는 어쩌면 '톰 크루즈'라는 배우의 영향력에 힘입어 흥행한 것일 수도 있습니다. 그리고 '크리스토퍼 놀란'이라는 감독은 워낙 영화계에 지대한 영향력을 비치기에 어떤영화를 만들든 장르를 불문하고 보는 사람들도 있습니다. 하지만 반대로 말하면, 매출 예측을 통해 주요 인기 원인과 홍보 전략을 세울 수 있습니다. 예를 들어 '다크 나이트'라는 영화가 생각보다 부진한 성적을 보인다고 가정하면, '크리스토퍼 놀란제작' 이라는 키워드를 내세워 적극적으로 홍보에 사용 할 수 있습니다.<br><bv>

- 셋째, 영화 마케팅의 확장
히로인 영화나, 귀여운 캐릭터가 등장하는 영화, 애니메이션 등등의 경우에도 매출에 따라 인기를 파악할 수 있으므로 굿즈 제작이나, 여러 먹을거리에 대한 프로모션 등등 다양한 방면으로 활용될 가능성이 있습니다.<br>

이 처럼 수요를 예측하여 여기에 맞는 비용을 산정하는 일은 매우 중요한 일이고 기업의 이윤을 추구하는 데 있어서 경제적인 일입니다.

그러나 아래에서 사용하는 데이터 셋들의 feature로 정확한 매출을 산정하는 것은 힘든일이기 때문에 영화제 수상 여부, 나라별 매출 등등의 요소들을 추가하여 개선해야할 필요가 있습니다.




In [1]:
from google.colab import files

uploaded = files.upload()

Saving movies (3).csv to movies (3).csv


In [73]:
# 사용한 라이브러리
import pandas as pd
import numpy as np
import re
import seaborn as sns

In [3]:
movie = pd.read_csv('movies (3).csv')

In [5]:
movie.head()

,Unnamed: 0,titles,years,imdp_ratings,meta_scores,votes,genres,directors,actors,movie_rates,grosses
0,0,Thor: Love and Thunder,(2022),6.8,57.0,129185.0,"\nAction, Adventure, Comedy",['TaikaWaititi'],"['ChrisHemsworth,NataliePortman,ChristianBale']",PG-13,[]
1,1,Top Gun: Maverick,(2022),8.6,78.0,245140.0,"\nAction, Drama",['JosephKosinski'],"['TomCruise,JenniferConnelly,MilesTeller']",PG-13,[]
2,2,The Black Phone,(2021),7.0,65.0,56374.0,"\nHorror, Thriller",['ScottDerrickson'],"['MasonThames,MadeleineMcGraw,EthanHawke']",R,[]
3,3,The Sea Beast,(2022),7.1,74.0,19035.0,"\nAnimation, Adventure, Comedy",['ChrisWilliams'],"['KarlUrban,Zaris-AngelHator,JaredHarris']",PG,[]
4,4,Elvis,(2022),7.8,64.0,48509.0,"\nBiography, Drama, Music",['BazLuhrmann'],"['TomHanks,AustinButler,OliviaDeJonge']",PG-13,[]


In [93]:

def data_eda(df):

  # Rank순으로 수집한 파일이므로 Unnamed: 0의 컬럼을 Rank로 바꾸고 1을 더합니다.
  df.rename(columns = {'Unnamed: 0' : 'rank'}, inplace=True)
  df['rank'] = df['rank'] + 1

  # Data의 쓰레기 값들을 지워줍니다.
  df['years'] = df['years'].replace('[\D+]', '', regex=True).astype('int')
  df['genres'] = df['genres'].replace('[\n]', '', regex=True)
  df['directors'] = df['directors'].replace("[\W]", '', regex=True)
  df['actors'] = df['actors'].replace("[\\[']", '', regex=True)
  df['actors'] = df['actors'].replace("\\]", '', regex=True)
  df['grosses'] = df['grosses'].replace('[\D]', '', regex=True)
  indexs = df[df['grosses'] == ''].index
  df['grosses'].iloc[indexs] = np.nan
  
  # 한 컬럼에 들어가있는 영화의 장르를 분리해 줍니다.
  genres = df.genres.str.split(',')
  df['genre1'] = genres.str[0]
  df['genre2'] = genres.str[1]
  df['genre3'] = genres.str[2]
  df.drop('genres',axis='columns',inplace=True)
  
  # 한 컬럼에 들어가있는 배우를 구분해 줍니다.
  actors = df.actors.str.split(',')
  df['lead_actor'] = actors.str[0]
  df['actor1'] = actors.str[1]
  df['actor2'] = actors.str[2]
  df.drop('actors',axis='columns', inplace=True)

  return df

In [92]:
movie_clean = data_eda(movie.copy())

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


ValueError: ignored

In [82]:
movie_clean.head()

,rank,titles,years,imdp_ratings,meta_scores,votes,genres,directors,movie_rates,grosses,genre1,genre2,genre3,lead_actor,actor1,actor2
0,1,Thor: Love and Thunder,2022,6.8,57.0,129185.0,"Action, Adventure, Comedy",TaikaWaititi,PG-13,NaN,Action,Adventure,Comedy,ChrisHemsworth,NataliePortman,ChristianBale
1,2,Top Gun: Maverick,2022,8.6,78.0,245140.0,"Action, Drama",JosephKosinski,PG-13,NaN,Action,Drama,NaN,TomCruise,JenniferConnelly,MilesTeller
2,3,The Black Phone,2021,7.0,65.0,56374.0,"Horror, Thriller",ScottDerrickson,R,NaN,Horror,Thriller,NaN,MasonThames,MadeleineMcGraw,EthanHawke
3,4,The Sea Beast,2022,7.1,74.0,19035.0,"Animation, Adventure, Comedy",ChrisWilliams,PG,NaN,Animation,Adventure,Comedy,KarlUrban,Zaris-AngelHator,JaredHarris
4,5,Elvis,2022,7.8,64.0,48509.0,"Biography, Drama, Music",BazLuhrmann,PG-13,NaN,Biography,Drama,Music,TomHanks,AustinButler,OliviaDeJonge


## Target값은 grosses 입니다. 단위는 Millon으로 최근 1~2년사이의 매출액 예측을 목표로합니다.

In [ ]:
movie_cl